In [1]:
from pickMFU import MFU_picker
import spacy
import jsonpickle
from transformers import BertTokenizer

# This load dictionaries for mfd, mfd2 and emfd
from emfdscore.load_mfds import * 


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/yu/miniconda3/envs/comp90055/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mfu_picker = MFU_picker(tokenizer=BertTokenizer.from_pretrained('bert-base-uncased'))

with open('./data/pickle_240227/voice.pickle', 'r') as f:
    voice_dict = jsonpickle.decode(f.read())
voice_keys = list(voice_dict.keys())

In [3]:
result_list = list()
for i in range(int(len(voice_keys))): # ~10 mins for the voice_dict
    key = voice_keys[i]
    content = voice_dict[key]
    text = content['text']
    
    # result = mfu_picker.pickMFU_mfd(text)
    # result = mfu_picker.pickMFU_emfd(text)
    result = mfu_picker.pickMFU(text, mfd_ver='mfd2')
    result_list.append(result)

In [6]:
# Inspect result
r = result_list[0]
for j in range(len(result_list)):
    r = result_list[j]
    for i in range(len(r[1])):
        moral_sent_index = r[1][i]
        moral_sent = r[0][moral_sent_index]
        moral_word_indices = r[2][i]
        for ind in moral_word_indices:
            word = moral_sent.split()[ind]

            # Test mfd output
            match = False
            # for v in mfd_regex.keys():
            #     if mfd_regex[v].match(word):
            #         match = True
            #         # print(word)
            # if not match:
            #     print('Sth is wrong')    

            # if word not in emfd:
            if word not in mfd2:
                print(f'Sth is wrong: {word}')
            else:
                # print(word)
                pass




In [7]:
for i in range(len(voice_keys)):
    key = voice_keys[i]
    voice_dict[key]['mfd2_bert_uncased'] = result_list[i]

save_path_template = "./data/pickle_240227/"

with open(save_path_template+'voice_m2bu.pickle', 'w') as voice_file:
    voice_file.write(jsonpickle.encode(voice_dict))

In [9]:
# For the testing of reading pickled content
voice_dict_loaded = None
with open('./data/pickle_240227/voice_mbu.pickle', 'r') as f:
    voice_dict_loaded = jsonpickle.decode(f.read())
voice_dict_loaded[voice_keys[7]]

{'text': 'The next few weeks and months will allow much of the necessary analysis to be done. However, there is no time like the present for key players in this dismal affair to take accountability and begin reform.\nThere is white-hot anger directed towards some of our big corporate entities – BHP, Rio Tinto, Wesfarmers, the big four banks and others – which, without consultation or consent from their shareholders, dived into the middle of a bitterly contested political dispute and poured millions of dollars of their shareholders’ money into partisan warfare.\nIt is bad enough that millions of dollars were wasted on a flawed campaign and misconceived voice model that was grossly unpopular with the overwhelming majority of Australians.\nThere is a "particular madness" to corporate Australia backing the Voice to Parliament, says Sky News host Amanda Stoker.\nThe participation of these big corporations was a significant factor in ensuring the Yes campaign was a resounding failure – voter

In [6]:
sent_len_list = list()
for i in range(len(voice_keys)):
    key = voice_keys[i]
    for sent in voice_dict_loaded[key]['mfd_bert_uncased'][0]:
        sent_len_list.append(len(sent.split()))

# voice_dict_loaded[voice_keys[7]]['mfd_bert_uncased'][0][3].split(" ")[24-5:24+5]

[42, 46, 69, 21, 49]

In [10]:
import numpy as np
np.percentile(sent_len_list, 75)

36.0

In [11]:
voice_dict_loaded[voice_keys[7]]['mfd_bert_uncased'][0][3]

'it is bad enough that millions of dollars were wasted on a flawed campaign and mis ##con ##ce ##ive ##d voice model that was gross ##ly unpopular with the overwhelming majority of australians .'